In [66]:
import pandas as pd
from datetime import datetime

In [114]:
def lineupstring(l):
    result = ""
    for i in eval(l):
        if i == 'brookins,vince':
            i = 'vincebrookins,jr'
        result += i +"; "
    return result

In [115]:
game1 = pd.read_csv("20181117.csv")
game2 = pd.read_csv("20181120.csv")
game3 = pd.read_csv("20181129.csv")
game4 = pd.read_csv("20181201.csv")
game5 = pd.read_csv("20181206.csv")
game6 = pd.read_csv("20181208.csv")

In [116]:
df = pd.concat([game1,game2,game3,game4,game5,game6], ignore_index=True)
#df.head()

In [117]:
df['Lineup'] = df['Lineup'].apply(lineupstring)
#df.head()

In [118]:
df['timex'] =  pd.to_datetime(df['timex'], format='%M:%S')
df['timey'] =  pd.to_datetime(df['timey'], format='%M:%S')
df['TotalTime'] = df['timex'].sub(df['timey'])
#df.head()

In [119]:
off_ppp = df.groupby('Lineup').aggregate({'Pts': 'sum',
                                'FGA': 'sum',
                                'OREB': 'sum',
                                'TOV': 'sum',
                                'FTA': 'sum',
                                'TotalTime' : 'sum'})

off_ppp['PPP'] = (off_ppp['Pts'] / ((off_ppp['FGA'] - off_ppp['OREB']) 
                                       + off_ppp['TOV'] + (0.44*off_ppp['FTA'])))
off_ppp.loc[off_ppp['TotalTime'] >= pd.Timedelta(5,'m')].sort_values(by = ('PPP'), ascending=False)

,Pts,FGA,OREB,TOV,FTA,TotalTime,PPP
Lineup,,,,,,,
"casey,bobby; heskett,james; kempton,michael; scadlock,kyle; soto,marcos;",26,12,4,2,8,00:08:20,1.923077
"babek,mickey; casey,bobby; heskett,james; kempton,michael; soto,marcos;",31,21,1,2,0,00:13:22,1.409091
"casey,bobby; feinberg,henry; heskett,james; karpowicz,matt; scadlock,kyle;",151,99,25,21,37,00:59:19,1.356937
"jones,jovan; kacmarek,daniel; moon,ryan; spivy,spencer; vincebrookins,jr;",10,7,3,3,2,00:06:00,1.269036
"babek,mickey; casey,bobby; heskett,james; karpowicz,matt; soto,marcos;",19,13,3,4,8,00:09:11,1.084475
"casey,bobby; feinberg,henry; heskett,james; kempton,michael; scadlock,kyle;",73,59,9,12,21,00:36:31,1.024705
"casey,bobby; heskett,james; karpowicz,matt; scadlock,kyle; taylor,marc;",12,7,1,4,4,00:05:29,1.020408
"babek,mickey; feinberg,henry; heskett,james; kempton,michael; scadlock,kyle;",8,6,2,3,5,00:05:31,0.869565


In [121]:
def_ppp = df.groupby('Lineup').aggregate({'opp_Pts': 'sum',
                                'opp_FGA': 'sum',
                                'opp_OREB': 'sum',
                                'opp_TOV': 'sum',
                                'opp_FTA': 'sum',
                                'TotalTime' : 'sum'})

def_ppp['PPP'] = (def_ppp['opp_Pts'] / ((def_ppp['opp_FGA'] - def_ppp['opp_OREB']) 
                                       + def_ppp['opp_TOV'] + (0.44*def_ppp['opp_FTA'])))
def_ppp.loc[def_ppp['TotalTime'] >= pd.Timedelta(5,'m')].sort_values(by = ('PPP'), ascending=True)

,opp_Pts,opp_FGA,opp_OREB,opp_TOV,opp_FTA,TotalTime,PPP
Lineup,,,,,,,
"jones,jovan; kacmarek,daniel; moon,ryan; spivy,spencer; vincebrookins,jr;",5,7,1,3,2,00:06:00,0.506073
"babek,mickey; feinberg,henry; heskett,james; kempton,michael; scadlock,kyle;",8,12,2,1,0,00:05:31,0.727273
"casey,bobby; heskett,james; kempton,michael; scadlock,kyle; soto,marcos;",12,10,1,7,1,00:08:20,0.729927
"casey,bobby; feinberg,henry; heskett,james; karpowicz,matt; scadlock,kyle;",84,99,9,12,14,00:59:19,0.776627
"babek,mickey; casey,bobby; heskett,james; karpowicz,matt; soto,marcos;",15,15,3,3,2,00:09:11,0.944584
"casey,bobby; feinberg,henry; heskett,james; kempton,michael; scadlock,kyle;",78,64,3,10,9,00:36:31,1.040555
"babek,mickey; casey,bobby; heskett,james; kempton,michael; soto,marcos;",24,18,1,4,4,00:13:22,1.054482
"casey,bobby; heskett,james; karpowicz,matt; scadlock,kyle; taylor,marc;",19,12,4,1,2,00:05:29,1.923077


In [74]:
#look up dictionary things dad -- go back and undo the lineup thing
test = 'casey,bobby; feinberg,henry; heskett,james; karpowicz,matt; scadlock,kyle; '

def undo(s):
    if s.find(';') != -1:
        return [s[:s.find(';')]] + undo(s[s.find(';')+2:]) 
    return []

for i in undo(test):
    print(i)

casey,bobby
feinberg,henry
heskett,james
karpowicz,matt
scadlock,kyle


In [75]:
roster = []
for i in df['Lineup'].apply(undo):
    for plyr in i:
        roster.append(plyr)
roster = list(set(roster))
roster

['jones,jovan',
 'babek,mickey',
 'taylor,marc',
 'vincebrookins,jr',
 'heskett,james',
 'soto,marcos',
 'kacmarek,daniel',
 'casey,bobby',
 'scadlock,kyle',
 'kempton,michael',
 'moon,ryan',
 'spivy,spencer',
 'feinberg,henry',
 'porath,jake',
 'karpowicz,matt']

In [144]:
df_three = df[df['Lineup'].str.contains('casey,bobby') & df['Lineup'].str.contains('scadlock,kyle') 
                         & df['Lineup'].str.contains('heskett,james')]
df_three.sum().TotalTime

Timedelta('0 days 01:58:35')

In [159]:
two_person_combos = []
three_person_combos = []
for i in range(len(roster)):
    for j in range(i+1,len(roster)):
        for k in range(j+1, len(roster)):
            three_player_dict = {}
            three_player_dict['name'] = [roster[i],roster[j],roster[k]]
            df_three = df[df['Lineup'].str.contains(roster[i]) & df['Lineup'].str.contains(roster[j]) & df['Lineup'].str.contains(roster[k])]
            df_three = df_three.sum()

            three_player_dict['Pts'] = df_three.Pts
            three_player_dict['FGA'] = df_three.FGA
            three_player_dict['OREB'] = df_three.OREB
            three_player_dict['TOV'] = df_three.TOV
            three_player_dict['FTA'] = df_three.FTA

            three_player_dict['opp_Pts'] = df_three.opp_Pts
            three_player_dict['opp_FGA'] = df_three.opp_FGA
            three_player_dict['opp_OREB'] = df_three.opp_OREB
            three_player_dict['opp_TOV'] = df_three.opp_TOV
            three_player_dict['opp_FTA'] = df_three.opp_FTA

            three_player_dict['TotalTime'] = pd.Timedelta(df_three.TotalTime)

            if pd.Timedelta(df_three.TotalTime) > pd.Timedelta(20,'m'):
                three_player_dict['Offensive PPP'] = df_three.Pts / ((df_three.FGA - df_three.OREB) 
                                                                    + df_three.TOV + (0.44*df_three.FTA))
                three_player_dict['Defensive PPP'] = df_three.opp_Pts / ((df_three.opp_FGA - df_three.opp_OREB) 
                                                                    + df_three.opp_TOV + (0.44*df_three.opp_FTA))
                three_person_combos.append(three_player_dict)
            
        two_player_dict = {}
        two_player_dict['name'] = [roster[i],roster[j]]
        df_two = df[df['Lineup'].str.contains(roster[i]) & df['Lineup'].str.contains(roster[j])]
        df_two = df_two.sum()
        
        two_player_dict['Pts'] = df_two.Pts
        two_player_dict['FGA'] = df_two.FGA
        two_player_dict['OREB'] = df_two.OREB
        two_player_dict['TOV'] = df_two.TOV
        two_player_dict['FTA'] = df_two.FTA
        
        two_player_dict['opp_Pts'] = df_two.opp_Pts
        two_player_dict['opp_FGA'] = df_two.opp_FGA
        two_player_dict['opp_OREB'] = df_two.opp_OREB
        two_player_dict['opp_TOV'] = df_two.opp_TOV
        two_player_dict['opp_FTA'] = df_two.opp_FTA
        
        two_player_dict['TotalTime'] = pd.Timedelta(df_two.TotalTime)
        
        if pd.Timedelta(df_two.TotalTime) > pd.Timedelta(20,'m'):
            two_player_dict['Offensive PPP'] = df_two.Pts / ((df_two.FGA - df_two.OREB) 
                                                                + df_two.TOV + (0.44*df_two.FTA))
            two_player_dict['Defensive PPP'] = df_two.opp_Pts / ((df_two.opp_FGA - df_two.opp_OREB) 
                                                                + df_two.opp_TOV + (0.44*df_two.opp_FTA))
            two_person_combos.append(two_player_dict)

In [160]:
two_player_df = pd.DataFrame(two_person_combos)
#two_player_df.loc[two_player_df['TotalTime'] >= pd.Timedelta(5,'m')]
two_player_df['Net PPP'] = two_player_df['Offensive PPP'] - two_player_df['Defensive PPP']
two_player_df.sort_values(by = ('Net PPP'), ascending=False)

,Defensive PPP,FGA,FTA,OREB,Offensive PPP,Pts,TOV,TotalTime,name,opp_FGA,opp_FTA,opp_OREB,opp_Pts,opp_TOV,Net PPP
0,0.482955,34,8,9,1.461814,49,5,00:20:23,"[jones,jovan, kacmarek,daniel]",28,5,3,17,8,0.978859
1,0.482955,34,8,9,1.461814,49,5,00:20:23,"[jones,jovan, moon,ryan]",28,5,3,17,8,0.978859
23,0.537634,34,10,10,1.497006,50,5,00:21:39,"[kacmarek,daniel, moon,ryan]",31,5,4,20,8,0.959372
2,0.689562,49,17,17,1.584786,80,11,00:31:28,"[babek,mickey, taylor,marc]",63,9,3,51,10,0.895224
12,0.818815,36,18,14,1.531180,55,6,00:24:04,"[taylor,marc, kempton,michael]",50,10,4,47,7,0.712365
8,0.654349,30,16,7,1.362398,60,14,00:27:14,"[babek,mickey, feinberg,henry]",50,9,7,34,5,0.708048
20,0.725953,30,14,7,1.393629,49,6,00:21:05,"[soto,marcos, scadlock,kyle]",33,7,1,32,9,0.667676
32,0.774436,120,42,26,1.389293,191,25,01:14:10,"[feinberg,henry, karpowicz,matt]",121,25,14,103,15,0.614857
21,0.759631,54,17,13,1.372354,83,12,00:36:51,"[soto,marcos, kempton,michael]",56,13,3,56,15,0.612723
19,0.863276,72,24,15,1.380814,114,15,00:45:30,"[soto,marcos, casey,bobby]",67,13,6,74,19,0.517538


In [161]:
three_player_df = pd.DataFrame(three_person_combos)
three_player_df['Net PPP'] = three_player_df['Offensive PPP'] - three_player_df['Defensive PPP']
three_player_df.sort_values(by = ('Net PPP'), ascending=False)

,Defensive PPP,FGA,FTA,OREB,Offensive PPP,Pts,TOV,TotalTime,name,opp_FGA,opp_FTA,opp_OREB,opp_Pts,opp_TOV,Net PPP
0,0.482955,34,8,9,1.461814,49,5,00:20:23,"[jones,jovan, kacmarek,daniel, moon,ryan]",28,5,3,17,8,0.978859
17,0.786424,110,39,25,1.382231,173,23,01:07:27,"[heskett,james, feinberg,henry, karpowicz,matt]",109,20,10,95,13,0.595807
23,0.794993,107,39,25,1.386596,168,22,01:05:15,"[casey,bobby, feinberg,henry, karpowicz,matt]",108,21,12,94,13,0.591603
25,0.764908,115,41,25,1.352977,180,25,01:11:19,"[scadlock,kyle, feinberg,henry, karpowicz,matt]",117,23,14,98,15,0.588069
18,0.829876,46,14,9,1.407349,72,8,00:29:31,"[soto,marcos, casey,bobby, kempton,michael]",41,11,3,48,15,0.577474
5,0.780624,35,5,8,1.290323,48,8,00:24:34,"[babek,mickey, soto,marcos, kempton,michael]",39,9,1,39,8,0.509698
7,0.906577,61,17,12,1.398917,93,10,00:37:36,"[heskett,james, soto,marcos, casey,bobby]",53,7,6,59,15,0.492340
15,0.878115,119,43,28,1.341357,185,28,01:13:57,"[heskett,james, scadlock,kyle, karpowicz,matt]",124,21,14,117,14,0.463243
8,0.865801,41,8,6,1.325247,59,6,00:25:41,"[heskett,james, soto,marcos, kempton,michael]",36,5,3,40,11,0.459446
6,0.846354,36,3,6,1.304802,50,7,00:22:33,"[babek,mickey, casey,bobby, kempton,michael]",35,7,2,39,10,0.458448
